# Analysing Toronto neighborhoods

This notebook is for scraping and analysing data of neighborhoods of Toronto. The first part is scraping the data from a Wikipedia page. The second part is to analyse the data to cluster the neighborhoods.

### 1. Scraping neighbourhood data from Wikipedia

The link to the Wikipedia page for Toronto neighborhoods is [here.](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

#### Scraping data using pandas :

In [1]:
import pandas as pd

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df_list = pd.read_html(url)

len(df_list)

3

In [3]:
df = df_list[0]
df.head()

Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

#### Scraping data using Beautiful Soup :

In [4]:
from bs4 import BeautifulSoup

import requests

In [6]:
import pprint
pp = pprint.PrettyPrinter()

page = requests.get(url)

soup = BeautifulSoup(page.content,'html.parser')

table = soup.find(id="bodyContent")

#elems = table.find_all(table,class_="wikitable sortable jquery-tablesorter")
#print(elems)

This section will be completed later. For now, we will use the pandas dataframe output.

### 2. Cleaning the data

The table needs to be cleaned to remove NA values.

In [7]:
df = df.drop(df[df['Borough']=="Not assigned"].index)
df.reset_index(inplace=True,drop=True)
df.head()

Postal Code           Borough                                Neighbourhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

On checking the data, there seems to be no duplicate entries of postal code.   
So, the final step in cleaning will be to replace NA values for Neighborhood with the Borough names.

In [8]:
for neighborhood in df["Neighbourhood"]:
    if neighborhood=="Not assigned":
        df.loc["Neighbourhood"]=df.loc["Borough"]
        
df.head()

Postal Code           Borough                                Neighbourhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [9]:
df.shape

(103, 3)

So we have 103 postal codes with complete information. Now, moving on to the analysis.

### 3. Adding Location data

The following code finds the latitude/longitude information of each postal code using Geocoder python, and builds a dataframe to include this information.

In [10]:
import geocoder

In [11]:
"""
count_calls = 0

for code in df["Postal Code"]:
    lat_lng_coords=None
    while(lat_lng_coords==None):
        g = geocoder.google('{}, Toronto, Ontario'.format(code))
        lat_lng_coords = g.latlng
        count_calls+=1
        
    df.loc["Latitude"] = lat_lng_coords[0]
    df.loc["Longitude"] = lat_lng_coords[1]
    
print("Total number of calls: ",count_calls,"\n") 
"""

df = df.sort_values(by="Postal Code").reset_index(drop=True)
df.head()

Postal Code      Borough                           Neighbourhood
0         M1B  Scarborough                          Malvern, Rouge
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2         M1E  Scarborough       Guildwood, Morningside, West Hill
3         M1G  Scarborough                                  Woburn
4         M1H  Scarborough                               Cedarbrae

The above code has not been successful, even after 2499 calls, as the package is said to be very unreliable.
So I will download the coordinates information from this [link.](https://cocl.us/Geospatial_data)

In [12]:
df_latlng = pd.read_csv("https://cocl.us/Geospatial_data")
df_latlng.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [13]:
df_latlng.shape

(103, 3)

I will add the coordinate information to the neighbourhoods dataframe.

In [14]:
df["Latitude"] = df_latlng["Latitude"]
df["Longitude"] = df_latlng["Longitude"]
df.head()

Postal Code      Borough                           Neighbourhood   Latitude  \
0         M1B  Scarborough                          Malvern, Rouge  43.806686   
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2         M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3         M1G  Scarborough                                  Woburn  43.770992   
4         M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

### 4. Segmenting and Clustering the Neighbourhoods

The aim of this section is to use Foursquare API to explore nearby venues for each neighbourhood cluster, or for each postal code. This data will be used to find out the most common venues for each location. And this information will be the basis of clustering the neighbourhoods of Toronto using K-Means clustering.    
First, I will import the required libraries.

In [15]:
import numpy as np
import json

from geopy.geocoders import Nominatim

import requests

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium

Using geopy for the coordinates of Toronto:

In [16]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.6534817, -79.3839347.


The following code displays a Map of Toronto with the neighborhoods as popup markers:

In [17]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Using FourSquare API

In [18]:
CLIENT_ID = *credentials hidden*
CLIENT_SECRET = *credentials hidden*

VERSION = '20200905'
LIMIT = 100
radius = 500

To cluster the neighborhoods, I am using Foursquare API to explore the nearby venues for each postal code and to display the count of venues for each postal code.

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
                  
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
                
        results = requests.get(url).json()["response"]['groups'][0]['items']
           
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],latitudes=df['Latitude'],
                                   longitudes=df['Longitude'])

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [21]:
toronto_venues.groupby("Neighborhood").count()

Neighborhood Latitude  \
Neighborhood                                                             
Agincourt                                                            5   
Alderwood, Long Branch                                               7   
Bathurst Manor, Wilson Heights, Downsview North                     21   
Bayview Village                                                      4   
Bedford Park, Lawrence Manor East                                   22   
...                                                                ...   
Willowdale, Willowdale West                                          5   
Woburn                                                               4   
Woodbine Heights                                                     8   
York Mills West                                                      2   
York Mills, Silver Hills                                             1   

                                                 Neighborhood Longitude  \
Neighborhood                                                              
Agincourt                                                             5   
Alderwood, Long Branch                                                7   
Bathurst Manor, Wilson Heights, Downsview North                      21   
Bayview Village                                                       4   
Bedford Park, Lawrence Manor East                                    22   
...                                                                 ...   
Willowdale, Willowdale West                                           5   
Woburn                                                                4   
Woodbine Heights                                                      8   
York Mills West                                                       2   
York Mills, Silver Hills                                              1   

                                                 Venue  Venue Latitude  \
Neighborhood                                                             
Agincourt                                            5               5   
Alderwood, Long Branch                               7               7   
Bathurst Manor, Wilson Heights, Downsview North     21              21   
Bayview Village                                      4               4   
Bedford Park, Lawrence Manor East                   22              22   
...                                                ...             ...   
Willowdale, Willowdale West                          5               5   
Woburn                                               4               4   
Woodbine Heights                                     8               8   
York Mills West                                      2               2   
York Mills, Silver Hills                             1               1   

                                                 Venue Longitude  \
Neighborhood                                                       
Agincourt                                                      5   
Alderwood, Long Branch                                         7   
Bathurst Manor, Wilson Heights, Downsview North               21   
Bayview Village                                                4   
Bedford Park, Lawrence Manor East                             22   
...                                                          ...   
Willowdale, Willowdale West                                    5   
Woburn                                                         4   
Woodbine Heights                                               8   
York Mills West                                                2   
York Mills, Silver Hills                                       1   

                                                 Venue Category  
Neighborhood                                                     
Agincourt                                                     5  
Alderwood, Long Branch                                        7 

The following code classifies the nearby venues of each postal code by category so that clustering by K Means can be performed.

In [22]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = list(toronto_onehot.columns[:])
fixed_columns.insert(0,fixed_columns.pop(fixed_columns.index("Neighborhood")))
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Neighborhood  Accessories Store  \
0                          Malvern, Rouge                  0   
1  Rouge Hill, Port Union, Highland Creek                  0   
2  Rouge Hill, Port Union, Highland Creek                  0   
3       Guildwood, Morningside, West Hill                  0   
4       Guildwood, Morningside, West Hill                  0   

   Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0                  0        0                   0             0   
1                  0        0                   0             0   
2                  0        0                   0             0   
3                  0        0                   0             0   
4                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   ...  Train Station  Turkish Restaurant  Vegetarian / Vegan Restaurant  \
0  ...              0                   0                              0   
1  ...              0                   0                              0   
2  ...              0                   0                              0   
3  ...              0                   0                              0   
4  ...              0                   0                              0   

   Video Game Store  Vietnamese Restaurant  Warehouse Store  Wine Bar  \
0                 0                      0                0         0   
1                 0                      0                0         0   
2                 0                      0                0         0   
3                 0                      0                0         0   
4                 0                      0                0         0   

   Wings Joint  Women's Store  Yoga Studio  
0            0              0            0  
1            0              0            0  
2            0              0            0  
3            0              0            0  
4            0              0            0  

[5 rows x 273 columns]

In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

The following code returns the 10 most common venues for each postal code.

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Agincourt                Lounge   
1                           Alderwood, Long Branch           Pizza Place   
2  Bathurst Manor, Wilson Heights, Downsview North                  Bank   
3                                  Bayview Village                  Café   
4                Bedford Park, Lawrence Manor East        Sandwich Place   

  2nd Most Common Venue      3rd Most Common Venue 4th Most Common Venue  \
0          Skating Rink  Latin American Restaurant        Clothing Store   
1              Pharmacy                        Gym        Sandwich Place   
2           Coffee Shop        Fried Chicken Joint    Chinese Restaurant   
3   Japanese Restaurant         Chinese Restaurant                  Bank   
4    Italian Restaurant                Coffee Shop      Greek Restaurant   

  5th Most Common Venue 6th Most Common Venue    7th Most Common Venue  \
0        Breakfast Spot           Event Space     Ethiopian Restaurant   
1           Coffee Shop                   Pub                  Dog Run   
2           Bridal Shop        Sandwich Place                    Diner   
3        Discount Store   Distribution Center                  Dog Run   
4       Thai Restaurant          Liquor Store  Comfort Food Restaurant   

  8th Most Common Venue      9th Most Common Venue 10th Most Common Venue  
0           Escape Room          Electronics Store                  Diner  
1    Dim Sum Restaurant                      Diner         Discount Store  
2            Restaurant  Middle Eastern Restaurant            Supermarket  
3      Doner Restaurant                 Donut Shop            Yoga Studio  
4             Juice Bar                    Butcher                   Café

#### K-Means clustering

K Means clustering is performed on the data and a column Cluster Labels is added to the dataframe to associate each postal code to each cluster. The clustering is performed by initializing number of clusters to 4.

In [26]:
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [27]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged = toronto_merged.dropna()
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype('int32')

toronto_merged.head()

Postal Code      Borough                           Neighbourhood   Latitude  \
0         M1B  Scarborough                          Malvern, Rouge  43.806686   
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2         M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3         M1G  Scarborough                                  Woburn  43.770992   
4         M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  Cluster Labels       1st Most Common Venue  \
0 -79.194353               1        Fast Food Restaurant   
1 -79.160497               1  Construction & Landscaping   
2 -79.188711               1              Breakfast Spot   
3 -79.216917               1                 Coffee Shop   
4 -79.239476               1        Caribbean Restaurant   

   2nd Most Common Venue 3rd Most Common Venue        4th Most Common Venue  \
0    Dumpling Restaurant        Discount Store          Distribution Center   
1                    Bar           Yoga Studio  Eastern European Restaurant   
2             Restaurant     Electronics Store          Rental Car Location   
3  Korean BBQ Restaurant    Mexican Restaurant          Dumpling Restaurant   
4       Hakka Restaurant                Bakery                  Gas Station   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0               Dog Run      Doner Restaurant            Donut Shop   
1               Dog Run      Doner Restaurant            Donut Shop   
2        Medical Center          Intersection                  Bank   
3   Distribution Center               Dog Run      Doner Restaurant   
4    Athletics & Sports                  Bank       Thai Restaurant   

  8th Most Common Venue        9th Most Common Venue 10th Most Common Venue  
0             Drugstore  Eastern European Restaurant          Women's Store  
1             Drugstore          Dumpling Restaurant      Electronics Store  
2    Mexican Restaurant                  Yoga Studio                Dog Run  
3            Donut Shop                    Drugstore            Yoga Studio  
4   Fried Chicken Joint             Doner Restaurant    Distribution Center

Finally, I can create a map of Toronto showing the neighbourhoods as part of different clusters shown by different colours.

In [28]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 5. Examining Clusters and Conclusion

Cluster 1:

In [29]:
toronto_merged.loc[toronto_merged["Cluster Labels"]==0,toronto_merged.columns[[2]+list(range(6,16))]]

Neighbourhood 1st Most Common Venue  \
14  Milliken, Agincourt North, Steeles East, L'Amo...                  Park   
23                                    York Mills West                  Park   
25                                          Parkwoods                  Park   
40      East Toronto, Broadview North (Old East York)                  Park   
44                                      Lawrence Park                  Park   
50                                           Rosedale                  Park   
74                                Caledonia-Fairbanks                  Park   
79           North Park, Maple Leaf Park, Upwood Park                  Park   
98                                             Weston                  Park   

         2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
14                  Playground                Bakery           Yoga Studio   
23           Convenience Store           Yoga Studio   Dumpling Restaurant   
25           Food & Drink Shop           Yoga Studio             Drugstore   
40           Convenience Store          Intersection           Yoga Studio   
44                 Swim School              Bus Line           Yoga Studio   
50                  Playground                 Trail           Yoga Studio   
74               Women's Store                  Pool             Drugstore   
79  Construction & Landscaping                Bakery           Yoga Studio   
98                 Yoga Studio   Dumpling Restaurant        Discount Store   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
14   Dumpling Restaurant   Distribution Center               Dog Run   
23        Discount Store   Distribution Center               Dog Run   
25        Discount Store   Distribution Center               Dog Run   
40             Drugstore        Discount Store   Distribution Center   
44             Drugstore   Distribution Center               Dog Run   
50             Drugstore                 Diner        Discount Store   
74                 Diner        Discount Store   Distribution Center   
79   Dumpling Restaurant   Distribution Center               Dog Run   
98   Distribution Center               Dog Run      Doner Restaurant   

   8th Most Common Venue 9th Most Common Venue       10th Most Common Venue  
14      Doner Restaurant            Donut Shop                    Drugstore  
23      Doner Restaurant            Donut Shop                    Drugstore  
25      Doner Restaurant            Donut Shop  Eastern European Restaurant  
40               Dog Run      Doner Restaurant                   Donut Shop  
44      Doner Restaurant            Donut Shop          Dumpling Restaurant  
50   Distribution Center               Dog Run             Doner Restaurant  
74               Dog Run      Doner Restaurant                   Donut Shop  
79      Doner Restaurant            Donut Shop                    Drugstore  
98            Donut Shop             Drugstore  Eastern European Restaurant

Cluster 2:

In [30]:
toronto_merged.loc[toronto_merged["Cluster Labels"]==1,toronto_merged.columns[[2]+list(range(6,16))]]

Neighbourhood  \
0                                       Malvern, Rouge   
1               Rouge Hill, Port Union, Highland Creek   
2                    Guildwood, Morningside, West Hill   
3                                               Woburn   
4                                            Cedarbrae   
..                                                 ...   
97                                    Humberlea, Emery   
99                                           Westmount   
100  Kingsview Village, St. Phillips, Martin Grove ...   
101  South Steeles, Silverstone, Humbergate, Jamest...   
102                Northwest, West Humber - Clairville   

          1st Most Common Venue  2nd Most Common Venue 3rd Most Common Venue  \
0          Fast Food Restaurant    Dumpling Restaurant        Discount Store   
1    Construction & Landscaping                    Bar           Yoga Studio   
2                Breakfast Spot             Restaurant     Electronics Store   
3                   Coffee Shop  Korean BBQ Restaurant    Mexican Restaurant   
4          Caribbean Restaurant       Hakka Restaurant                Bakery   
..                          ...                    ...                   ...   
97               Baseball Field            Yoga Studio   Distribution Center   
99                  Pizza Place            Coffee Shop        Discount Store   
100                 Pizza Place                   Park        Sandwich Place   
101                 Pizza Place          Grocery Store              Pharmacy   
102               Garden Center    Rental Car Location                   Bar   

           4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0            Distribution Center               Dog Run      Doner Restaurant   
1    Eastern European Restaurant               Dog Run      Doner Restaurant   
2            Rental Car Location        Medical Center          Intersection   
3            Dumpling Restaurant   Distribution Center               Dog Run   
4                    Gas Station    Athletics & Sports                  Bank   
..                           ...                   ...                   ...   
97                       Dog Run      Doner Restaurant            Donut Shop   
99                Sandwich Place    Chinese Restaurant          Intersection   
100                     Bus Line            Donut Shop        Discount Store   
101          Fried Chicken Joint        Sandwich Place  Fast Food Restaurant   
102                    Drugstore      Doner Restaurant                 Diner   

    7th Most Common Venue        8th Most Common Venue  \
0              Donut Shop                    Drugstore   
1              Donut Shop                    Drugstore   
2                    Bank           Mexican Restaurant   
3        Doner Restaurant                   Donut Shop   
4         Thai Restaurant          Fried Chicken Joint   
..                    ...                          ...   
97              Drugstore          Dumpling Restaurant   
99    Dumpling Restaurant  Eastern European Restaurant   
100   Distribution Center                      Dog Run   
101            Beer Store                 Liquor Store   
102        Discount Store          Distribution Center   

           9th Most Common Venue 10th Most Common Venue  
0    Eastern European Restaurant          Women's Store  
1            Dumpling Restaurant      Electronics Store  
2                    Yoga Studio                Dog Run  
3                      Drugstore            Yoga Studio  
4               Doner Restaurant    Distribution Center  
..                           ...                    ...  
97   Eastern European Restaurant                  Field  
99                     Drugstore           Dessert Shop  
100             Doner Restaurant              Drugstore  
101       Gluten-free Restaurant       Department Store  
102                      Dog Run             Donut Shop  

[89 rows x 11 columns]

Cluster 3:

In [31]:
toronto_merged.loc[toronto_merged["Cluster Labels"]==2,toronto_merged.columns[[2]+list(range(6,16))]]

Neighbourhood 1st Most Common Venue 2nd Most Common Venue  \
20  York Mills, Silver Hills   Martial Arts School           Yoga Studio   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
20   Dumpling Restaurant        Discount Store   Distribution Center   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
20               Dog Run      Doner Restaurant            Donut Shop   

   9th Most Common Venue       10th Most Common Venue  
20             Drugstore  Eastern European Restaurant

Cluster 4:

In [32]:
toronto_merged.loc[toronto_merged["Cluster Labels"]==3,toronto_merged.columns[[2]+list(range(6,16))]]

Neighbourhood 1st Most Common Venue  \
94  West Deane Park, Princess Gardens, Martin Grov...            Print Shop   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
94   Dumpling Restaurant        Discount Store   Distribution Center   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
94               Dog Run      Doner Restaurant            Donut Shop   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
94             Drugstore           Yoga Studio     Dim Sum Restaurant

From examining the 4 clusters, I can classify them as given below:

1. Cluster 1: Neighbourhoods where the most frequented spots are parks and recreational centers. These neighbourhoods could be considered having the most active residents. So this will be an Active cluster.

2. Cluster 2: Neighbourhoods where most frequented places are all the types of restaurants among naturally common spots like gas stations and pharmacies. Most of the neighbourhoods come under this cluster, which is natural, as there wont be a standout factor which differentiates them from the rest. Also, many fast food joints like Pizza Place come up, showing the general food loving nature of the residents. This can be the Foodie cluster / The Common cluster.

3. Cluster 3: Only couple of neighbourhoods show up in this cluster, having a varied interest in venues like Martial Arts and Yoga. Even the restaurants are exotic choices. So I can classify this cluster as Exotic.

4. Cluster 4: The last cluster has a few neighbourhoods with Print shop as the most frequented spot among other restaurants and stores. It is possible that there are many public offices requiring lots of printed applications or reports. So I can classify it as a Business cluster.

Finally, the analysis of the various neighbourhoods of Toronto using FourSquare data has been completed. The principle behind this analysis was to identify different neighbourhoods based on the most commonly referred venues at that location.    
It can be now safely said, that a study of this data can bring out several inferences like the lifestyle of the residents, their passions and even what political factors would be considered important by the residents of each neighbourhood. The analysis of data opens up many opportunities for governments, businesses and policymakers to plan for the future in the very best and efficient way.